In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from random import random
from scipy.optimize import minimize
from gurobipy import *

In [2]:
x = sp.symbols('x')
cap = 2000 # total emission cap
Regulator8 = Regulator("test1", permit_price = 2.4079, emission_cap = cap)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator8)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0, regulator= Regulator8)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0, regulator= Regulator8)
country1 = Country('DE', 1, regulator= Regulator8)
country2 = Country('FI', 0.5, regulator= Regulator8)
country3 = Country('GR', size= 0.1, regulator= Regulator8)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 10*x+ 2*x**2 + 0.1*x**3 , 0, 0, 0, regulator= Regulator8)
firm2 = Firm('firm2', 1, 2, x*0, 11*x+ 3*x**2 + 0.2*x**3, 0, 0, 0, regulator= Regulator8)
firm3 = Firm('firm3', 1, 3, x*0, 5*x+ 4*x**2 + 5*x**3 , 0, 0, 0, regulator= Regulator8)
firm4 = Firm('firm4', 2, 1, x*0, 7*x+ 5*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator8)
firm5 = Firm('firm5', 2, 2, x*0, 1*x+ 6*x**2 + 2*x**3 , 0, 0, 0, regulator= Regulator8)
firm6 = Firm('firm6', 2, 3, x*0, 2*x+ 7*x**2 + 3*x**3 , 0, 0, 0, regulator= Regulator8)
firm7 = Firm('firm7', 3, 1, x*0, 3*x+ 8*x**2 + 4*x**3 , 0, 0, 0, regulator= Regulator8)
firm8 = Firm('firm8', 3, 2, x*0, 4*x+ 9*x**2 + 10*x**3 , 0, 0, 0, regulator= Regulator8)
firm9 = Firm('firm9', 3, 3, x*0, 5*x+ 10*x**2 + 11*x**3 , 0, 0, 0, regulator= Regulator8)



In [3]:
# Regulator8.find_optimal_permit_price_to_meet_the_emission_cap_requirements(precision=0.01)
Regulator8.optimization_with_least_squares(gurobi_print= False, BAU = True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14
Optimal solution found
Firm firm1 has output 0 and emission 0
Firm firm2 has output 0 and emission 0
Firm firm3 has output 0 and emission 0
Firm firm4 has output 0 and emission 0
Firm firm5 has output 0 and emission 0
Firm firm6 has output 0 and emission 0
Firm firm7 has output 0 and emission 0
Firm firm8 has output 0 and emission 0
Firm firm9 has output 0 and emission 0
Permit price: 0.0


<gurobi.Model MIP instance Least Squares: 22 constrs, 51 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [ ]:
# Let's see what happens in we change the free allocation of sector A keeping the cap at 80% of the BAU emissions
free_allocation_sectorA = list(np.linspace(0, 1, 100))
# Read the data from the csv file, if it exists, if not, just create a new one
try:
    df = pd.read_csv('../data/generated/exp04_Cap_vs_permit_price.csv')
    free_alloc = list(df['Free Allocation'])
    caps2 = list(df['Emission Cap'])
    permit_prices = list(df['Permit Price'])
    outputs = list(df['Outputs'])
    emissions = list(df['Emissions'])
    profits = list(df['Profits'])
except:
    free_alloc = []
    caps2 = []
    permit_prices = []
    outputs = []
    emissions = []
    profits = []
# Loop through all the emission caps and calculate the permit price

# Find all the caps that are not calculated yet (meaning that they are more than 0.5 different than a value of the caps2)
for i in free_allocation_sectorA[:]:  # Iterate over a copy of the list
    if any(abs(i - x) < 0.001 for x in free_alloc):
        free_allocation_sectorA.remove(i)

for i in tqdm.tqdm(free_allocation_sectorA):
    Regulator8.emission_cap = i*Regulator8.BAU_emissions
    Regulator8.optimization_with_least_squares(gurobi_print= False)
    caps2.append(Regulator8.emission_cap)
    free_alloc.append(i)
    permit_prices.append(Regulator8.permit_price)
    outputs.append(tuple([firm.actual_output for firm in Regulator8.firm_registry.values()]))
    emissions.append(tuple([firm.emission for firm in Regulator8.firm_registry.values()]))
    profits.append(tuple([firm.profit for firm in Regulator8.firm_registry.values()]))
    # save all the data in a csv file
    df = pd.DataFrame({'Free Allocation': free_alloc, 'Emission Cap': caps2, 'Permit Price': permit_prices, 'Outputs': outputs, 'Emissions': emissions, 'Profits': profits})
    # Save the DataFrame to the specified path
    df.to_csv('../data/generated/exp04_Cap_vs_permit_price.csv', index=False)

  1%|          | 1/100 [00:13<22:55, 13.89s/it]

Optimal solution found
Firm firm1 has output 11.590294724722236 and emission 0.0
Firm firm2 has output 7.969434183487138 and emission 0.0
Firm firm3 has output 2.230751148812899 and emission 0.0
Firm firm4 has output 3.1034682114644427 and emission 0.0
Firm firm5 has output 3.6494971947133408 and emission 0.0
Firm firm6 has output 2.9954734023246274 and emission 0.0
Firm firm7 has output 2.852492693370358 and emission 0.0
Firm firm8 has output 1.533895172825282 and emission 0.0
Firm firm9 has output 1.4397552515562961 and emission 0.0
Permit price: 96.66192252182557
